### Python 代码运行并测试运行时间

In [ ]:
import numpy as np
from time import time

rng = np.random.default_rng()


def measure_runtime(func, *args, **kwargs):
    start_time = time()
    result = func(*args, **kwargs)
    runtime = time() - start_time  
    return {'result': result, 'runtime': runtime}


def RandomData(n, p):
    mean_vec = np.zeros(p)
    cov_mat = 0.5 ** np.abs(np.arange(1, p + 1).reshape(-1, 1) - np.arange(1, p + 1))
    X = rng.multivariate_normal(mean_vec, cov_mat, n, method="cholesky")
    return X


def MatrixMultiplication(p):
    a = rng.normal(size=(p, p))
    b = a.T @ a
    return b


def DistributedRandom(n, p, m):
    mean_vec = np.zeros(p)
    cov_mat = 0.5 ** np.abs(np.arange(1, p + 1).reshape(-1, 1) - np.arange(1, p + 1))
    x = [0] * m
    for i in range(m):
        x[i] = rng.multivariate_normal(mean_vec, cov_mat, n, method="cholesky")
    return x


# Example usage:
Time_data = measure_runtime(RandomData, n=200, p=2000)
print(f"随机数生成时间: {Time_data['runtime']} seconds")

Time_matrix = measure_runtime(MatrixMultiplication, p=2000)
print(f"矩阵乘法时间: {Time_matrix['runtime']} seconds")


Distributed_time = measure_runtime(DistributedRandom, n=6000, p=600, m=60)
print(f"分布式随机数生成时间: {Distributed_time['runtime']} seconds")


随机数生成结果: (200, 2000)
随机数生成时间: 0.08186912536621094 seconds
矩阵乘法结果: (2000, 2000)
矩阵乘法时间: 0.07513713836669922 seconds
分布式随机数生成时间: 3.210928201675415 seconds


In [ ]:
# Metrics for method results
def method_metrics(method_beta, beta_true):
    beta_index1 = np.where(beta_true != 0)[0]
    beta_index2 = np.where(method_beta != 0)[0]
    SC = int(np.all(np.isin(beta_index1, beta_index2)))  # Set intersection check for SC
    CF = int( np.array_equal(beta_index1, beta_index2) )    # Check if both sets are equal for CF
    AMS = len(beta_index2)   # Active Model Size (number of non-zero coefficients in method_beta)
    PSR = len(np.intersect1d(beta_index2, beta_index1)) / len(beta_index1)  # Precision
    FDR = len(np.setdiff1d(beta_index2, beta_index1)) / len(beta_index2)  # False Discovery Rate
    ME = np.linalg.norm(method_beta - beta_true)  # Mean error (Euclidean distance)
    
    return np.array([SC, CF, AMS, PSR, FDR, ME])